<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/TP1_PARTE3_M6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Java (Sockets)**
Comunicar dos procesos no emparentados a través del paso de mensajes mediante sockets.
Existirá un programa “Cliente” que:
  • Tomará por parámetro una cadena de caracteres, por ejemplo; “Hirschfeldt2023”.
  • Mostrará por pantalla los siguientes resultados:
    Cantidad de caracteres totales
    Cantidad de letras
    Cantidad de dígitos
    Cantidad de caracteres que no son letra ni dígito.
Existirá otro programa “Servidor” que:
  • Será el encargado de generar y enviar la      información requerida por el programa cliente

In [ ]:
%%writefile Server.java
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.ServerSocket;
import java.net.Socket;

public class Server 
{
    public static void main(String[] args) throws IOException 
    {
        int port = 1234;
        ServerSocket server = new ServerSocket(port);
        System.out.println("Server started on port " + port);
        
        while (true) 
        {
            Socket client = server.accept();
            System.out.println("Client connected from " + client.getInetAddress().getHostName());
            
            // Read the message sent by the client
            BufferedReader in = new BufferedReader(new InputStreamReader(client.getInputStream()));
            String message = in.readLine();
            
            // Generate the required information
            int totalChars = message.length();
            int letters = message.replaceAll("[^a-zA-Z]", "").length();
            int digits = message.replaceAll("[^0-9]", "").length();
            int nonAlphanumeric = totalChars - letters - digits;
            
            // Send the information to the client
            PrintWriter out = new PrintWriter(client.getOutputStream(), true);
            out.println(totalChars);
            out.println(letters);
            out.println(digits);
            out.println(nonAlphanumeric);
            
            client.close();
            System.out.println("Client disconnected");
        }
    }
}


Writing Server.java


In [ ]:
%%writefile Client.java
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.PrintWriter;
import java.net.Socket;

public class Client 
{
    public static void main(String[] args) throws IOException 
    {
        String host = "localhost";
        int port = 1234;
        
        // Get the message from command line parameters
        String message = args[0];
        
        Socket server = new Socket(host, port);
        
        // Send the message to the server
        PrintWriter out = new PrintWriter(server.getOutputStream(), true);
        out.println(message);
        
        BufferedReader in = new BufferedReader(new InputStreamReader(server.getInputStream()));
        int totalChars = Integer.parseInt(in.readLine());
        int letters = Integer.parseInt(in.readLine());
        int digits = Integer.parseInt(in.readLine());
        int nonAlphanumeric = Integer.parseInt(in.readLine());
        
        System.out.println("Total characters: " + totalChars);
        System.out.println("Letters: " + letters);
        System.out.println("Digits: " + digits);
        System.out.println("Non-alphanumeric characters: " + nonAlphanumeric);
        
        server.close();
    }
}


Writing Client.java


In [ ]:
!javac Server.java && javac Client.java

In [ ]:
!nohup java Server 1>salida-servidor 2>/dev/null & 

In [ ]:
!cat salida-servidor

Server started on port 1234


In [ ]:
!java Client Password.1

Total characters: 10
Letters: 8
Digits: 1
Non-alphanumeric characters: 1


In [ ]:
!cat salida-servidor

Server started on port 1234
Client connected from localhost
Client disconnected


# **C/C++**

Cliente - recibe cadena de numeros, muestra resultado por pantalla
NOTA: intento hacer threads servidor y cliente porque no se como ejecutar dos programas al mismo tiempo

In [ ]:
%%writefile cliente.cpp
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <iostream>
#include <sstream>
#include <pthread.h>
#include <thread>

#define MAX_NUMEROS 4
using namespace std;

int validarCantidadNumeros(const string &s, const char delimiter)
{
  int cantNum=0,numero;
  string token;
  stringstream ss(s);
  while (getline(ss, token, delimiter))
  {
    try
    {
      numero = std::stoi(token);
      if(numero < 0)
      {
        return -1;
      }
    }
    catch (const std::invalid_argument& e)
    {
      return -1;
    }
    cantNum++;
  }
  return cantNum;
}

int entradaValida(int argc, string entrada)
{
  if(argc != 2)
  {
    return -1;
  }
  int cantNumeros = validarCantidadNumeros(entrada,'-');
  if(cantNumeros > MAX_NUMEROS || cantNumeros == -1)
  {
    return -1;
  }
  return 0;
}

void cliente(string entrada)
{
  if( mkfifo("./fifo",0666) == -1 )
	{
		perror("Error al crear ./fifo");
		pthread_exit(NULL);
	}
  cout << "open fifo" << endl;
  int fifoWrite = open("./fifo", O_WRONLY);
  cout << "write fifo" << endl;
  if( write( fifoWrite, &entrada, entrada.length() ) == -1 )
	{
		perror("Error en write()");
		pthread_exit(NULL);
	}
  cout << "close fifo" << endl;	
	close(fifoWrite);	
  cout << "unlik fifo" << endl;
	unlink("./fifo");
  cout << "exit" << endl;
  pthread_exit(NULL);
}

void servidor()
{
  std::this_thread::sleep_for(std::chrono::seconds(100));
  string entrada;
  int fifoRead = open("./fifo", O_RDONLY);
  if( read( fifoRead, &entrada, sizeof( string ) ) == -1 )
	{
		perror("Error en read()");
		pthread_exit(NULL);
	}
  cout<< "el servidor recibio esta entrada: " << entrada << endl;
  close(fifoRead);
  pthread_exit(NULL);
}

int main (int argc, char *argv[])
{
  string entrada = argv[1];
  if(entradaValida(argc,entrada) == -1)
  {
    perror("Entrada invalida - Debe ser una cadena con una serie de cuatro números enteros no negativos separados con guión medio (-) ");
		return EXIT_FAILURE;
  }
  std::thread t_cliente(cliente,entrada);
  std::thread t_servidor(servidor);
  t_cliente.join();
  t_servidor.join();
	return EXIT_SUCCESS;
}

Overwriting cliente.cpp


Servidor - espera que el cliente mande por medio de fifo la entrada para procesar los calculos y devolver resultados

In [ ]:
%%writefile servidor.cpp
#include <stdlib.h>
#include <stdio.h>
#include <string.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <iostream>
#include <sstream>

using namespace std;

int main (int argc, char *argv[])
{
  string entrada;
  int fifoRead = open("./fifo", O_RDONLY);
  if( read( fifoRead, &entrada, sizeof( string ) ) == -1 )
	{
		perror("Error en read()");
		return EXIT_FAILURE;
	}
  cout<< entrada << endl;
  close(fifoRead);	
	return EXIT_SUCCESS;
}


Overwriting servidor.cpp


In [ ]:
!g++ -std=c++11 -pthread cliente.cpp -o cliente

In [ ]:
!g++ servidor.cpp -o servidor

In [ ]:
!ls -l

total 64
-rwxr-xr-x 1 root root 55616 May  3 20:08 cliente
-rw-r--r-- 1 root root  2163 May  3 20:08 cliente.cpp
drwxr-xr-x 1 root root  4096 May  1 13:36 sample_data


In [ ]:
!rm fifo

In [ ]:
!./cliente 1-2-3

open fifo
write fifo
close fifo
unlik fifo
exit
el servidor recibio esta entrada: 
free(): invalid pointer


In [ ]:
!./servidor